In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk("./"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Frame the problem
Using the customer description, Define the problem your trying to solve in your own words (remember this is not technial but must be specific so the customer understands the project

The goal of this project is to train a computer to play the game Tetris on its own. Instead of programming the computer with fixed rules, I want it to learn through experience. The computer will observe the game state, try different moves, and learn which choices help it survive longer and clear more rows. Over time, the system should improve and become able to score higher without any human input.

# 2. Get the Data 
Define how you recieved the data (provided, gathered..)

Before the model starts learning it needs example gameplay results. The Tetris environment was already provided so I used the Random agent and Greedy agent to play several Tetris games where I recorded how many pieces were placed before losing and how many rows were cleared. This data will give us a baseline performance to compare against later on our model.

In [8]:
import csv
from game import Game

with open("tetris_data.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["episode", "pieces_dropped", "rows_cleared"])

    for episode in range(10):
        g = Game("random")
        pieces, rows = g.run_no_visual()
        writer.writerow([episode, pieces, rows])
        print(f"Episode {episode}: pieces={pieces}, rows={rows}")

print("Data saved to tetris_data.csv")


Episode 0: pieces=17, rows=0
Episode 1: pieces=17, rows=0
Episode 2: pieces=24, rows=0
Episode 3: pieces=20, rows=0
Episode 4: pieces=15, rows=0
Episode 5: pieces=20, rows=0
Episode 6: pieces=19, rows=0
Episode 7: pieces=25, rows=0
Episode 8: pieces=15, rows=0
Episode 9: pieces=17, rows=0
Data saved to tetris_data.csv


In [6]:
import os, sys, csv

os.chdir("/home/jupyter-1006929/my_tetris_project")
sys.path.append(os.path.join(os.getcwd(), "src"))

from game import Game

output_file = "tetris_data2.csv"

with open(output_file, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["episode", "pieces_dropped", "rows_cleared"])  

    for episode in range(10):   
        print(f"--- Running Game {episode} ---")
        g = Game("greedy")      
        pieces, rows = g.run_fast()   
        print(f"Result: {pieces}, {rows}")

        writer.writerow([episode, pieces, rows])

print(f"\nData saved to {output_file}")

--- Running Game 0 ---
Result: 233, 78
--- Running Game 1 ---
Result: 12891, 5364
--- Running Game 2 ---
Result: 5383, 2235
--- Running Game 3 ---
Result: 684, 267
--- Running Game 4 ---
Result: 5143, 2129
--- Running Game 5 ---
Result: 2046, 842
--- Running Game 6 ---
Result: 946, 378
--- Running Game 7 ---
Result: 12238, 5093
--- Running Game 8 ---
Result: 25869, 10756
--- Running Game 9 ---
Result: 26259, 10915

Data saved to tetris_data2.csv


# 3. Explore the Data
Gain insights into the data you have from step 2, making sure to identify any bias

After collecting gameplay results from the random Tetris agent, the dataset was analyzed to understand its performance. The data showed that the agent dropped between 14 and 24 pieces per game, with an average of around 20.3 pieces. However, the agent cleared zero rows in every game, indicating that it was not attempting to score points at all.

Because row clearing never occurred, there was no variation in the rows_cleared column, and therefore no meaningful correlation could be computed between pieces dropped and rows cleared. This confirms that the random agent’s behavior is purely survival-based and not strategic. The slight variation in the number of pieces dropped simply reflects randomness in board placement rather than intentional play. This will serve as a baseline for the creation of my actual model.

The greedy agent dropped between 233 and 26,259 pieces per game and cleared between 78 and 10,915 rows, with averages of 9,169 pieces and 3,806 rows cleared. This shows that it is actually thinking and making strategic moves as it should. There is almost a perfect correlation between pieces placed and rows cleared, so this confirms this will work as a good baseline for our actual model.

In [4]:
import pandas as pd

df = pd.read_csv("tetris_data.csv")
print(df)

print("\nSummary Statistics:")
print(df.describe())

print("\nCorrelation Matrix:")
print(df.corr())


   episode  pieces_dropped  rows_cleared
0        0              19             0
1        1              19             0
2        2              23             0
3        3              21             0
4        4              21             0
5        5              19             0
6        6              24             0
7        7              14             0
8        8              23             0
9        9              20             0

Summary Statistics:
        episode  pieces_dropped  rows_cleared
count  10.00000       10.000000          10.0
mean    4.50000       20.300000           0.0
std     3.02765        2.869379           0.0
min     0.00000       14.000000           0.0
25%     2.25000       19.000000           0.0
50%     4.50000       20.500000           0.0
75%     6.75000       22.500000           0.0
max     9.00000       24.000000           0.0

Correlation Matrix:
                 episode  pieces_dropped  rows_cleared
episode         1.000000       -0.0063

In [7]:
import pandas as pd

data = pd.read_csv("tetris_data2.csv")

print("\nData Preview:")
print(data.head())

print("\nSummary Statistics:")
print(data.describe())

print("\nCorrelation Matrix:")
print(data.corr())


Data Preview:
   episode  pieces_dropped  rows_cleared
0        0             233            78
1        1           12891          5364
2        2            5383          2235
3        3             684           267
4        4            5143          2129

Summary Statistics:
        episode  pieces_dropped  rows_cleared
count  10.00000       10.000000     10.000000
mean    4.50000     9169.200000   3805.700000
std     3.02765     9972.043118   4152.154302
min     0.00000      233.000000     78.000000
25%     2.25000     1221.000000    494.000000
50%     4.50000     5263.000000   2182.000000
75%     6.75000    12727.750000   5296.250000
max     9.00000    26259.000000  10915.000000

Correlation Matrix:
                 episode  pieces_dropped  rows_cleared
episode         1.000000        0.656991      0.656758
pieces_dropped  0.656991        1.000000      0.999999
rows_cleared    0.656758        0.999999      1.000000


# 4.Prepare the Data


Apply any data transformations and explain what and why


To prepare the data I applied a normalization step because the two values we are using our measured on different scales are pieces dropped range from 17-26, while rows cleared is usually 0-1. To prevent this imbalance, I used Min-Max normalization,which rescales both features to a scale from 0 to 1.

In [5]:
import pandas as pd

# Load data
df = pd.read_csv("tetris_data.csv")

def min_max_normalize(col):
    return (col - col.min()) / (col.max() - col.min())

df["pieces_dropped_norm"] = min_max_normalize(df["pieces_dropped"])
df["rows_cleared_norm"] = min_max_normalize(df["rows_cleared"])

print("\nNormalized Data:")
print(df[["episode", "pieces_dropped_norm", "rows_cleared_norm"]])

df.to_csv("tetris_data_normalized.csv", index=False)

For this project, I plan to build a Reinforcement Learning model to play Tetris. Unlike the greedy agent the RL model learns from experience by interacting with the game and receiving rewards for good moves. This approach is better suited for Tetris because strong play  often requires long-term planning, such as arranging pieces to clear multiple lines later instead of only focusing on immediate placement. I will probably use a DQN, which allows the agent to estimate which moves will lead to better results by analyzing the board state and improving those decisions over many played games.

# 5. Model the data
Using selected ML models, experment with your choices and describe your findings. Finish by selecting a Model to continue with


# 6. Fine Tune the Model

With the select model descibe the steps taken to acheve the best rusults possiable 


# 7. Present
In a customer faceing Document provide summery of finding and detail approach taken


# 8. Launch the Model System
Define your production run code, This should be self susficent and require only your model pramaters 
